# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [3]:
# import the requests library (1 line)

import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)

url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)

status_url = "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)

req = requests.get(url+status_url)

# check the status_code using a condition and print appropriate messages (4 lines)

if req.status_code not in [400, 401, 404, 500]:
    print(f"Status Code: {req.status_code}")
else:
    print(f"Request was not succesful")

"""
try: 
    print(f"Status Code: {req.status_code}")
    #print(f"Response: {req.text[:100]}...")  ## Print first 100 characters
    #print(f"Request was successful: {req.ok}")
except:
    print(f"Request was not succesful")
"""


Status Code: 200


'\ntry: \n    print(f"Status Code: {req.status_code}")\n    #print(f"Response: {req.text[:100]}...")  ## Print first 100 characters\n    #print(f"Request was successful: {req.ok}")\nexcept:\n    print(f"Request was not succesful")\n'

### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [4]:
import json

# Set the countries_url variable (1 line)

countries_url = "https://country-leaders.onrender.com"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)

req = requests.get(countries_url+"/countries", headers = {'accept': 'application/json'})

# Get the JSON content and store it in the countries variable (1 line)

countries = req.json() 

# display the request's status code and the countries variable (1 line)

print(f"Status code: {req.status_code} {countries}")

Status code: 403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [5]:
# Set the cookie_url variable (1 line)

cookie_url ="https://country-leaders.onrender.com"

# Query the enpoint, set the cookies variable and display it (2 lines)

req = requests.get(cookie_url+"/cookie")

cookies = req.cookies
print (cookies)

<RequestsCookieJar[<Cookie user_cookie=b804c12a-2f05-4f29-9164-095503bbc657 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [6]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

req = requests.get(countries_url+"/countries", cookies=cookies)

# display the countries variable (1 line)

countries = req.json()
print(countries)


['fr', 'be', 'ma', 'us', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [7]:
# Set the leaders_url variable (1 line)

leaders_url = "https://country-leaders.onrender.com"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

leaders = requests.get(leaders_url + "/leaders")

# display the leaders variable (1 line)

print(leaders)
# 403 Forbidden

<Response [403]>


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [8]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)

req = requests.get(leaders_url+"/leaders", cookies=cookies, params={'country':'fr'})

# assign the output to the leaders variable (1 line)

leaders = req.json()

# display the leaders variable (1 line)

print(leaders)


[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [9]:
# 4 lines
leaders_per_country ={}
for country in countries:
    req = requests.get(leaders_url+"/leaders", cookies=cookies, params={'country': country})
    leaders = req.json()
    leaders_per_country[country] = leaders
print(leaders_per_country)

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

In [10]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [11]:
# < 15 lines
def get_leaders():
    url="https://country-leaders.onrender.com"
    endpoints = ['/status', '/countries', '/cookie', '/leaders']
    if requests.get(url+endpoints[0]):
        req = requests.get(url+endpoints[2])
        cookies = req.cookies
        requests.get(url+endpoints[1], cookies=cookies)
        countries = req.json()
        leaders_per_country ={}
        for country in countries:
            req = requests.get(url+"/leaders", cookies=cookies, params={'country': country})
            leaders = req.json()
            leaders_per_country[country] = leaders
    else:
        print('Not succesful connection')
    return leaders_per_country
        
    

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [12]:
# 2 lines
print(leaders_per_country)

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [13]:
# 3 lines
#from bs4 import BeautifulSoup as bs
import requests

url = 'https://en.wikipedia.org/wiki/Fran%C3%A7ois_Hollande'
req = requests.get(url)
#soup = bs(req.text,"lxml")
for line in req.text.splitlines():
    print (line)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>François Hollande - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-f

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [5]:
# 3 lines
from bs4 import BeautifulSoup as bs
soup = bs(req.text,"lxml")
print(soup.get_text())
#print(soup.prettify())

{"message":"The cookie is missing"}


That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [15]:
# 2 lines
#paragraphs = [p_tag.get_text() for p_tag in soup.select(".unique-phrase p")]
#print(paragraphs)

for el in soup.find_all("p"):
    print (el.get_text())
#    for elem2 in elem.find_all("div", attrs={"class":"content-txt"}):
#print(paragraph)

paragraph = soup.find_all("p")





François Gérard Georges Nicolas Hollande (French: [fʁɑ̃swa ʒeʁaʁ ʒɔʁʒ(ə) nikɔla ɔlɑ̃d] ⓘ; born 12 August 1954) is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party (PS) from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.

Born in Rouen and raised in Neuilly-sur-Seine, Hollande began his political career as a special advisor to newly elected President François Mitterrand before serving as a staffer for Max Gallo the government's spokesman. He became a member of the National Assembly in 1988 and was elected First Secretary of the PS in 1997. Following the 2004 regional elections won by the PS, Hollande was cited as a potential presidential candidate, but he res

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [16]:
# <10 lines
#first_paragraph = paragraph[2]
#print(first_paragraph.get_text())

#consider regex with the listening sign ⓘ

for el in soup.find_all("p"):
    paragraph_element = el.get_text()
    for el2 in paragraph_element:
        if el2 == 'ⓘ':
            first_paragraph = el
            print(first_paragraph.get_text())
            break


François Gérard Georges Nicolas Hollande (French: [fʁɑ̃swa ʒeʁaʁ ʒɔʁʒ(ə) nikɔla ɔlɑ̃d] ⓘ; born 12 August 1954) is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party (PS) from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [17]:
# 10 lines
# def get_first_paragraph(wikipedia_url):

#from bs4 import BeautifulSoup as bs

def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook

    print(wikipedia_url)

#   [insert your code]
    req = requests.get(wikipedia_url)
    soup = bs(req.text,"lxml")

    #paragraph = soup.find_all("p")
    #first_paragraph = paragraph[2].get_text()
    
    for el in soup.find_all("p"):
        paragraph_element = el.get_text()
        for el2 in paragraph_element:
            if el2 == 'ⓘ':
                first_paragraph = el.get_text()
                #print(first_paragraph.get_text())
                break
  
#   return first_paragraph
    return first_paragraph

In [18]:
# Test: 3 lines
#get_first_paragraph('https://en.wikipedia.org/wiki/Andrew_Jackson')
get_first_paragraph('https://en.wikipedia.org/wiki/Martin_Van_Buren')

https://en.wikipedia.org/wiki/Martin_Van_Buren


"Martin Van Buren (/væn ˈbjʊərən/ van BYOO-rən; Dutch: Maarten van Buren [ˈmaːrtə(ɱ) vɑm ˈbyːrə(n)] ⓘ; December 5, 1782 – July 24, 1862) was the eighth president of the United States, serving from 1837 to 1841. A primary founder of the Democratic Party, he served as New York's attorney general and U.S. senator, then briefly as the ninth governor of New York before joining Andrew Jackson's administration as the tenth United States secretary of state, minister to the United Kingdom, and ultimately the eighth vice president from 1833 to 1837, after being elected on Jackson's ticket in 1832. Van Buren won the presidency in 1836 against divided Whig opponents. He lost re-election in 1840, and failed to win the Democratic nomination in 1844. Later in his life, he re-emerged as an elder statesman and an anti-slavery leader who led the Free Soil Party ticket in the 1848 presidential election.\n"

### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [19]:
# 3 lines
import re

string = get_first_paragraph('https://en.wikipedia.org/wiki/Martin_Van_Buren')
# ([\(\[]).*?([\)\]]) to delete what's inside brackets and parenthesis

parameter = r'([\[]).*?([\]])|[ⓘ]'
#["'\'", 'ⓘ','[', ']']
#r'[\ⓘ[]]'

re.sub(parameter, '', string)


https://en.wikipedia.org/wiki/Martin_Van_Buren


"Martin Van Buren (/væn ˈbjʊərən/ van BYOO-rən; Dutch: Maarten van Buren  ; December 5, 1782 – July 24, 1862) was the eighth president of the United States, serving from 1837 to 1841. A primary founder of the Democratic Party, he served as New York's attorney general and U.S. senator, then briefly as the ninth governor of New York before joining Andrew Jackson's administration as the tenth United States secretary of state, minister to the United Kingdom, and ultimately the eighth vice president from 1833 to 1837, after being elected on Jackson's ticket in 1832. Van Buren won the presidency in 1836 against divided Whig opponents. He lost re-election in 1840, and failed to win the Democratic nomination in 1844. Later in his life, he re-emerged as an elder statesman and an anti-slavery leader who led the Free Soil Party ticket in the 1848 presidential election.\n"

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines
def regex_applied(wikipedia_url):

    #print(wikipedia_url)
    global first_paragraph
    
    req = requests.get(wikipedia_url)
    soup = bs(req.text,"lxml")
    
    paragraph = soup.find_all("p")
    #first_paragraph = paragraph[2].get_text()
    
    #identify first paragraph with regex:
    parameter = r'([\[]).*?([\]])|[ⓘ]'
    
   
    for el in soup.find_all("p"):
        paragraph_element = el.get_text()
        for el2 in paragraph_element:
            if el2 == 'ⓘ':
                first_paragraph = el.get_text()
                #print(first_paragraph.get_text())
                break
            else:
                first_paragraph = paragraph[2].get_text()
                break
    
    string = first_paragraph
    replaced_text = re.sub(parameter, '', string)
    return replaced_text


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines
regex_applied('https://en.wikipedia.org/wiki/Martin_Van_Buren')

'On the television show Seinfeld, the 1997 episode "The Van Buren Boys" is about a fictional street gang that admires Van Buren and bases its rituals and symbols on him, including the hand sign of eight fingers pointing up to signify Van Buren, the eighth president.\n'

## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [34]:
# < 20 lines
def get_leaders():
    url="https://country-leaders.onrender.com"
    endpoints = ['/status', '/countries', '/cookie', '/leaders']
    if requests.get(url+endpoints[0]):
        cookie_req = requests.get(url+endpoints[2])
        cookies = cookie_req.cookies
        countries_req=requests.get(url+endpoints[1], cookies=cookies)
        countries = countries_req.json()
        leaders_per_country ={}
        for country in countries:
            leaders_req = requests.get(url+endpoints[3], cookies=cookies, params={'country': country})
            leaders = leaders_req.json()
         
            # Open wikipedia_url and Add first paragraph in dictionary
            for leader in leaders:
                #print(leader)
                #print(leader['wikipedia_url'])
                #url = leader['wikipedia_url']
                leader["first_paragraph"] = regex_applied(leader['wikipedia_url'])
                #leaders.append(leader)
                
            leaders_per_country[country] = leaders
            
    else:
        print('Not succesful connection')
    
    #
 

In [13]:
# Check the output of your function (2 lines)
print(get_leaders())

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14', 'first_paragraph': '\n'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15', 'first_paragraph': "«\xa0Les médias nous avaient déjà fait le coup avec les partisans du «\xa0non\xa0» à la Constitution européenne en les décrivant comme des moins que rien, des zombies. C'est une des raisons qui m'aurait poussé à voter «\xa0non\xa0», si je n'avais pas été politiquement obligé de me prononcer pour le «\xa0oui\xa0». Car les Français peuvent avoir une opinion tranchée, mais vouloir, en même temps, que le pré

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [8]:
# < 25 lines

def get_leaders():
    url="https://country-leaders.onrender.com"
    endpoints = ['/status', '/countries', '/cookie', '/leaders']
    if requests.get(url+endpoints[0]):
        cookie_req = requests.get(url+endpoints[2])
        cookies = cookie_req.cookies
        countries_req=requests.get(url+endpoints[1], cookies=cookies)
        countries = countries_req.json()
        leaders_per_country ={}
        for country in countries:
            try:
                leaders_req = requests.get(url+endpoints[3], cookies=cookies, params={'country': country})
                leaders = leaders_req.json()
         
                # Open wikipedia_url and Add first paragraph in dictionary
                for leader in leaders:
                    leader["first_paragraph"] = regex_applied(leader['wikipedia_url'])
                    #leaders.append(leader)
    
                leaders_per_country[country] = leaders     
            except  Exception as e:
                # requests.exceptions.HTTPError as errh
                cookie_req = requests.get(url + endpoints[2])
                cookies = cookie_req.cookies
                continue 
    else:
            print('Not succesful connection')
    return leaders_per_country
    
   

Check the output of your function again.

In [24]:
# Check the output of your function (1 line)
print(get_leaders())

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14', 'first_paragraph': "François Hollande  Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.\n"}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15', 'first_paragraph': '\n'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [30]:
# < 20 lines
from requests import Session

def first_paragraph(wikipedia_url, session):

    #print(wikipedia_url)
    #global first_paragraph
    
    #session = requests.Session()
    req = session.get(wikipedia_url)
    soup = bs(req.text,"lxml")
    
    paragraph = soup.find_all("p")
    #first_paragraph = paragraph[2].get_text()
    
    #identify first paragraph with regex:
    parameter = r'([\[]).*?([\]])|[ⓘ]'
    
   
    for el in soup.find_all("p"):
        paragraph_element = el.get_text()
        for el2 in paragraph_element:
            if el2 == 'ⓘ':
                first_paragraph = el.get_text()
                #print(first_paragraph.get_text())
                break
            else:
                first_paragraph = paragraph[2].get_text()
                break
    
    string = first_paragraph
    first_paragraph = re.sub(parameter, '', string)
    return first_paragraph

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [31]:
# <25 lines
def get_leaders():
    url="https://country-leaders.onrender.com"
    endpoints = ['/status', '/countries', '/cookie', '/leaders']
    
    session = requests.Session()  # Main session for API
    wiki_session = requests.Session()  # Separate session for Wikipedia
    
    if session.get(url+endpoints[0]):
        cookie_req = session.get(url+endpoints[2])
        cookies = cookie_req.cookies
        countries_req=session.get(url+endpoints[1], cookies=cookies)
        countries = countries_req.json()
        leaders_per_country ={}
    
        for country in countries:
            try:
                leaders_req = session.get(url+endpoints[3], cookies=cookies, params={'country': country})
                leaders = leaders_req.json()
         
                # Open wikipedia_url and Add first paragraph in dictionary
                for leader in leaders:
                    leader["first_paragraph"] = first_paragraph(leader['wikipedia_url'], wiki_session)
                    #leaders.append(leader)
    
                leaders_per_country[country] = leaders     
            except  Exception as e:
                # requests.exceptions.HTTPError as errh
                cookie_req = session.get(url + endpoints[2])
                cookies = cookie_req.cookies
                continue 
    else:
            print('Not succesful connection')
    return leaders_per_country
    

Test your new functions.



In [32]:
print(get_leaders())

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14', 'first_paragraph': "François Hollande  Écouter, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.\n"}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15', 'first_paragraph': '\n'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [12]:
# 3 lines

dictionary = get_leaders()

# Serializing json
json_object = json.dumps(dictionary, indent=4)

# Writing to sample.json
with open("leaders_per_country.json", "w") as outfile:
    outfile.write(json_object)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [11]:
# 3 lines

# Open and read the JSON file
with open('leaders_per_country.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data)

{}


Make a function `save(leaders_per_country)` to call this code easily.

In [33]:
# 3 lines
def save(leaders_per_country):
    dictionary = get_leaders()

    # Serializing json
    json_object = json.dumps(dictionary, indent=4)

    # Writing to sample.json
    with open("leaders_per_country.json", "w") as outfile:
        outfile.write(json_object)

In [34]:
# Call the function (1 line)
save(get_leaders())


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!